## Step 2 - Climate App

#### Design a Flask API based on the queries saved in 'climate_starter.ipynb'
 
* Routes
       * `/`
           * Home page.
           * List all routes that are available.
       * `/api/v1.0/precipitation`
           * Convert the query results to a Dictionary using `date` as the key and `prcp` as the value.
           * Return the JSON representation of your dictionary.
       * `/api/v1.0/stations`
           * Return a JSON list of stations from the dataset.
       * `/api/v1.0/tobs`
           * Query the dates and temperature observations of the most active station for the last year of data.
           * Return a JSON list of Temperature Observations (tobs) for the previous year
       * `/api/v1.0/<start>` and `/api/v1.0/<start>/<end>`
           * Return a JSON list of the minimum temperature, the average temperature, and
               the max temperature for a given start or start-end range.
           * When given the start only, calculate `TMIN`, `TAVG`, and `TMAX` for all dates 
               greater than and equal to the start date.
           * When given the start and the end date, calculate the `TMIN`, `TAVG`, and `TMAX` 
               for dates between the start and end date inclusive.

In [18]:
# Import Dependencies 
import pandas as pd
import numpy as np

import datetime as dt

import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func

from flask import Flask, jsonify

In [19]:
# Database setup
engine = create_engine("sqlite:///Resources/hawaii.sqlite")

# Reflect an existing database into a new model
Base = automap_base()

# Reflect the tables
Base.prepare(engine, reflect=True)

# Save reference to the tables
Measurement = Base.classes.measurement
Station = Base.classes.station

# Create our session (link) from Python to the DB
session = Session(engine)

# Flask Setup:
app = Flask(__name__)

In [20]:
# Home page.
# List all routes that are available.

@app.route("/")
def home_page():
    return (
        f"<div align='center'><h1>Climate App</h1><br/></div>"
        f"<br/>"
        f"/api/v1.0/precipitation<br/>"
        f"<br/>"
        f"/api/v1.0/stations<br/>"
        f"<br/>"
        f"/api/v1.0/tobs<br/>"
        f"<br/>"
        f"/api/v1.0/<start><br/>"
        f"Put the start date in 'YYYY-MM-DD' format<br/>"
        f"<br/>"
        f"/api/v1.0/<start>/<end><br/>"   
        f"Put the dates in 'YYYY-MM-DD/YYYY-MM-DD' format<br/>"
        )

In [21]:
# Convert the query results to a dictionary using date as the key and prcp as the value.
# Return the JSON representation of your dictionary.

@app.route("/api/v1.0/precipitation")  
def prcp():
    prev_year = dt.date(2017,8,23) - dt.timedelta(days = 365)
    start_date = session.query(Measurement.date).order_by(Measurement.date.desc()).first()
    prcp = session.query(Measurement.date, Measurement.prcp).\
                                       filter(Measurement.date <= End_date).\
                                       filter(Measurement.date >= Start_date).all()                                                                  

    list = []
    for i in prcp:
        dict ={"Date:":prcp[0], "Precipitation:":prcp[1]}
        list.append(dict)
    return jsonify(list)

In [22]:
## Return a JSON list of stations from the dataset.
@app.route("/api/v1.0/stations")
def stations():
    stations = session.query(Station.station,Station.name).all()
    
    list=[]
    for station in stations:
        dict = {"Station ID:":stations[0],"Station Name":stations[1]}
        list.append(dict)
    return jsonify(list)

In [23]:
# Query the dates and temperature observations of the most active station for the last year of data.
# Return a JSON list of Temperature Observations (tobs) for the previous year.

@app.route("/api/v1.0/tobs")
def tobs():
    prev_year = dt.date(2017, 8, 23) - dt.timedelta(days=365)
    tobs = session.query(Measurement.date, Measurement.tobs).\
    filter(Measurement.station == 'USC00519281', Measurement.date >= prev_year).all()
    
    list = []
    for temp in tobs:
        dict = {'date': temp[0], "tobs": temp[1]}
        list.append(dict)
    return jsonify(list)

In [24]:
# Return a JSON list of the minimum temperature, the average temperature, and the max temperature for 
# a given start or start-end range.
# When given the start only, calculate TMIN, TAVG, and TMAX for all dates greater than and equal to the start date.
# When given the start and the end date, calculate the TMIN, TAVG, and TMAX for dates between the start and end date inclusive.

@app.route("/api/v1.0/<start>")
def temp_start(start):
    results = session.query(func.min(Measurement.tobs), func.avg(Measurement.tobs), func.max(Measurement.tobs)).\
              filter(Measurement.date >= start).order_by(Measurement.date.desc()).all()

    list = []
    print(f"Temperature Analysis for the dates greater than or equal to the start date")
    for temps in results:
        dict = {"Minimum Temp":results[0][0],"Average Temp":results[0][1],"Maximum Temp":results[0][2]}
        list.append(dict)
    return jsonify(list)

In [25]:
@app.route("/api/v1.0/<start>/<end>")
def start_end(start,end):
    results = session.query(func.min(Measurement.tobs), func.avg(Measurement.tobs),func.max(Measurement.tobs)).\
                  filter(Measurement.date >= start, Measurement.date <= end).order_by(Measurement.date.desc()).all()
    print(f"Temperature Analysis for the dates greater than or equal to the start date and lesser than or equal to the end date")
    for temps in results:
        dict = {"Minimum Temp":results[0][0],"Average Temp":results[0][1],"Maximum Temp":results[0][2]}
    return jsonify(dict)   

In [ ]:
if __name__ == "__main__":
    app.run(debug=True)